In [22]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import time
import keras
from load_cifar import *
from keras.datasets import cifar10

In [23]:
folder = "cifar-10-batches-py"
preprocess_data(folder)

<h1>Define Hyper-perparmeter</h1>

In [114]:
hidden_size1 = 1024
hidden_size2 = 1024
output_size = 10
lr = 0.0002
epoch = 20
batch_size = 32
batch_num = 5
dropout_rate = 0.5

<h1>Define Placeholder</h1>

In [115]:
tf.reset_default_graph()
with tf.name_scope("data"):
    X = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 3])
    y = tf.placeholder(dtype=tf.float32, shape=[None, 10])
    training = tf.placeholder_with_default(False, shape=())

<h1>Define Neural Network Architecture</h1>

In [116]:
def neuralNet(X):
    regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
    X_reshape = tf.layers.flatten(X)
    fc1 = tf.layers.dense(X_reshape, hidden_size1, activation=tf.nn.relu, use_bias=True)#, kernel_regularizer=regularizer)
    #layer1_drop = tf.layers.dropout(fc1, dropout_rate,  training=training)
    fc2 =  tf.layers.dense(fc1, hidden_size2, activation=tf.nn.sigmoid, use_bias=True)#, kernel_regularizer=regularizer)
    layer2_drop = tf.layers.dropout(fc2, dropout_rate,  training=training)
    #fc3 =  tf.layers.dense(fc2, hidden_size3, activation=tf.nn.sigmoid, use_bias=True)
    #fc4 =  tf.layers.dense(fc3, hidden_size3, activation=tf.nn.sigmoid, use_bias=True)
    out =  tf.layers.dense(layer2_drop, output_size, activation=tf.nn.softmax, use_bias=True)
    return out
out = neuralNet(X)

<h1>Define cost andoptimization</h1>

In [117]:
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=out)
correct_pred = tf.equal(tf.argmax(out,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [118]:
start = time.time()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for i in range(epoch):
        for j in range(1, batch_num + 1):
            file_name = 'preprocess_batch_' + str(j) + '.p'
            data = pickle.load(open(file_name, 'rb'))
            features, labels = data[0], data[1]
            tr_acc = 0
            count = 0
            for batch_x, batch_y in mini_batch(features, labels, batch_size):
                sess.run(train_op, feed_dict={X:batch_x, y:batch_y, training: True})
                if j == batch_num:
                    tr_acc += sess.run(accuracy,feed_dict={X:batch_x, y:batch_y})
                    count += 1
            if j == batch_num:
                print("epoch "+str(i)+", Training accuracy= {:.3f}".format(tr_acc / count))
        val_x, val_y = load_preprocessed_validation_batch()
        val_acc = sess.run(accuracy,feed_dict={X:val_x, y:val_y})
        print("epoch "+str(i)+", Validation accuracy= {:.3f}".format(val_acc))
        print("time elapsed at "+str(i)+"th epoch is: ", int(time.time() - start), " seconds")
    save_path = saver.save(sess, 'model.ckpt')
    print("Training finished!")
    acc_test = 0.0
    count = 0
    test_batch_size = 200
    for batch_x, batch_y in load_preprocessed_test_batch(test_batch_size):
            test_acc = sess.run(accuracy, feed_dict={X:batch_x, y:batch_y})
            acc_test += test_acc
            count += 1
    print("Test accuracy is: {:.3f}".format(acc_test / count))

epoch 0, Training accuracy= 0.333
epoch 0, Validation accuracy= 0.347
time elapsed at 0th epoch is:  9  seconds
epoch 1, Training accuracy= 0.365
epoch 1, Validation accuracy= 0.345
time elapsed at 1th epoch is:  19  seconds
epoch 2, Training accuracy= 0.385
epoch 2, Validation accuracy= 0.389
time elapsed at 2th epoch is:  28  seconds
epoch 3, Training accuracy= 0.408
epoch 3, Validation accuracy= 0.391
time elapsed at 3th epoch is:  37  seconds
epoch 4, Training accuracy= 0.422
epoch 4, Validation accuracy= 0.417
time elapsed at 4th epoch is:  46  seconds
epoch 5, Training accuracy= 0.436
epoch 5, Validation accuracy= 0.423
time elapsed at 5th epoch is:  55  seconds
epoch 6, Training accuracy= 0.444
epoch 6, Validation accuracy= 0.431
time elapsed at 6th epoch is:  64  seconds
epoch 7, Training accuracy= 0.452
epoch 7, Validation accuracy= 0.434
time elapsed at 7th epoch is:  73  seconds
epoch 8, Training accuracy= 0.465
epoch 8, Validation accuracy= 0.446
time elapsed at 8th epoch i

<h1>Analysis</h1>

In this problem, I split training set and validation set using the ratio(98:2) from the original training set. For the fully connected neural network. I first explore the shallow layers of 2, and search hidden size between 200 and 2000. I also try to increase the layer to 3 and even more, but for me I don't get better accuracy. The best accuracy I've got so far is using 2 layers with both layers the hidden size of 1024. After training 20 epoches, I get test accuracy of 0.476 for my test set. I try to ask others how they get an accuracy of 50% within 10 epoches and try their hyperparameter settings, but the parameter doesn't work for me. I believe there's some problem with my data preprocessing in part(a) that causes the inadequate accuracy.